In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Catboost
from catboost import CatBoostRegressor

#Sklearn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

#SkForecast
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.preprocessing import TimeSeriesDifferentiator

In [15]:
df= pd.read_csv("/Users/santi/Documents/csv/datos_arreglados.csv",sep=",",index_col=0)

df["date"]=pd.to_datetime(df["date"])

df.head(5)

,date,item,category,department,stores,region,week,year,is_holidays,units_sold,sell_price,total_sales
0,2013-07-08,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,28,2013,0,0,12.7414,0.0000
1,2013-07-15,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,28,2013,1,0,12.7414,0.0000
2,2013-07-22,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,29,2013,0,4,10.9858,43.9432
3,2013-07-29,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,30,2013,0,6,10.9858,65.9148
4,2013-08-05,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,31,2013,0,0,10.9858,0.0000


In [16]:
df_boston_1= df[df["stores"]== "South_End(BOS_1)"].copy()
df_boston_1.tail()

,date,item,category,department,stores,region,week,year,is_holidays,units_sold,sell_price,total_sales
6609638,2016-03-28,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,South_End(BOS_1),Boston,12,2016,1,3,1.2,3.6
6609639,2016-04-04,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,South_End(BOS_1),Boston,13,2016,0,6,1.2,7.2
6609640,2016-04-11,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,South_End(BOS_1),Boston,14,2016,0,17,1.2,20.4
6609641,2016-04-18,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,South_End(BOS_1),Boston,15,2016,0,20,1.2,24.0
6609642,2016-04-25,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,South_End(BOS_1),Boston,16,2016,0,5,1.2,6.0


In [17]:
df_boston_1fit = pd.pivot_table(
           data    = df_boston_1,
           values  = 'total_sales',
           index   = 'date',
           columns = 'item'
       )
df.set_index("date", inplace=True)
df_boston_1fit = df_boston_1fit.asfreq('1W')

In [18]:
df_boston_1fit.fillna(0, inplace=True)

In [19]:
df_boston_1fit.head(50)

item,ACCESORIES_1_001,ACCESORIES_1_002,ACCESORIES_1_003,ACCESORIES_1_004,ACCESORIES_1_005,ACCESORIES_1_006,ACCESORIES_1_007,ACCESORIES_1_008,ACCESORIES_1_009,ACCESORIES_1_010,...,SUPERMARKET_3_818,SUPERMARKET_3_819,SUPERMARKET_3_820,SUPERMARKET_3_821,SUPERMARKET_3_822,SUPERMARKET_3_823,SUPERMARKET_3_824,SUPERMARKET_3_825,SUPERMARKET_3_826,SUPERMARKET_3_827
date,,,,,,,,,,,,,,,,,,,,,
2011-02-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-02-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-02-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-02-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-03-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-03-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-03-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-03-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-04-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
pipeline = make_pipeline(TimeSeriesDifferentiator(order=1),StandardScaler())
pipeline

Pipeline(steps=[('timeseriesdifferentiator', TimeSeriesDifferentiator()),
                ('standardscaler', StandardScaler())])

In [24]:
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = CatBoostRegressor(random_state=123),
                 lags               = 12,
                 transformer_exog   = pipeline,
                 weight_func        = None,
                 series_weights     = None
             )

In [25]:
forecaster.fit(series=df_boston_1fit)

: 

In [13]:
forecasting_prediction_8= forecaster.predict(steps=8)

/Users/santi/anaconda3/envs/Forecasting/lib/python3.11/site-packages/skforecast/utils/utils.py:867: UserWarning: `last_window` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
/Users/santi/anaconda3/envs/Forecasting/lib/python3.11/site-packages/skforecast/utils/utils.py:867: UserWarning: `last_window` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
/Users/santi/anaconda3/envs/Forecasting/lib/python3.11/site-packages/skforecast/utils/utils.py:867: UserWarning: `last_window` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
/Users/santi/anaconda3/envs/Forecasting/lib/python3.11/site-packages/skforecast/utils/utils.py:867: UserWarning: `last_window` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
/Users/santi/anaconda3/envs/Forecasting/lib/python3.11/s

KeyboardInterrupt: 